In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("..")  # Go up one level to the UROP directory

In [2]:
import torch
from torch.utils.data import DataLoader
import yaml

import librosa
import matplotlib.pyplot as plt
%matplotlib inline

with open("config.yaml") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

batch_size = config["batch_size"]
lr = config["lr"]
num_epochs = config["num_epochs"]
num_workers = config["num_workers"]

In [3]:
from utils import LazyWindowedDataset, train_test_split

# Initialize Dataset
full_train_source_dataset = LazyWindowedDataset(
    root_dir="datasets/BrushlessMotor",
    split="train",
    anomaly_type=['normal'],
    domain_type=['source', 'target'],
    window_size_ms=100,
    stride_ms=50,
)

train_source_dataset, val_source_dataset = train_test_split(full_train_source_dataset)

train_loader = DataLoader(train_source_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
val_loader = DataLoader(val_source_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True)

In [4]:
test_source_dataset = LazyWindowedDataset(
    root_dir="datasets/BrushlessMotor",
    split="test",
    anomaly_type=['normal', 'anomaly'],
    domain_type=['source', 'target'],
    window_size_ms=100,
    stride_ms=50,
)
test_loader = DataLoader(test_source_dataset, batch_size=batch_size, shuffle=True, num_workers=1, drop_last=True)

In [5]:
from models import Autoencoder
from utils import train_one_epoch, evaluate, train_model, z_score_normalize, flatten_and_concat

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Autoencoder()
model.to(device)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [6]:
evaluate(model, test_loader, criterion)

{'normal', 'magnetic', 'belt'}


(0.9990665986243472, np.float64(0.530181623931624))

In [7]:
train_model(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=num_epochs,
    verbose=False,
)

{'normal'}
Epoch [1/2] | Train Loss: 0.840433 | Val Loss: 0.818828 | Val AUC: nan


/home/massimiliano/Projects/UROP/utils/train.py:81: UserWarning: Only one class present in y_true. ROC AUC score is undefined.
  val_auc = float('nan')


KeyboardInterrupt: 

In [8]:
evaluate(model, test_loader, criterion)

(0.8663650898344488, np.float64(0.578690006574622))